# Audio Modeling comparision between
- Whisper
- Kaldi Vosk
- Kaldi Wav2Vec 2.0
- Hugging Face

In [1]:
#===========================
# Load preprocessed audio
#===========================
import soundfile as sf

audio_path = r"E:\3rd YEAR CSE(AI-ML)\5thsem\Machine Learning\ISMAIL\processed_audio.wav"
audio, sr = sf.read(audio_path)

print("Sample rate:", sr)
print("Duration (sec):", len(audio)/sr)

Sample rate: 16000
Duration (sec): 4.08


In [2]:
from IPython.display import Audio
print("Actual Aduio:")
display(Audio(audio, rate=sr))

Actual Aduio:


In [3]:
#ffmpeg checking for whisper-model
import subprocess
subprocess.run(["ffmpeg", "-version"])

CompletedProcess(args=['ffmpeg', '-version'], returncode=0)

In [4]:
#Clearing cache to use GPU efficiently
import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [5]:
#===========================
# Model 1 — Whisper (ASR) 
#===========================
import whisper

whisper_model = whisper.load_model("small")
whisper_out = whisper_model.transcribe(audio_path)
whisper_text = whisper_out["text"]
print("Whisper Output:")
print(whisper_text)


Whisper Output:
 What's the biggest misunderstanding about you? Whenever you hear about somebody who's...


In [6]:
#===================================
# Model 2 — Kaldi alternative (Vosk)
#===================================
from vosk import Model, KaldiRecognizer
import json

vosk_model = Model(lang="en-us")
rec = KaldiRecognizer(vosk_model, sr)

with open(audio_path, "rb") as f:
    f.read(44)   # skip wav header
    rec.AcceptWaveform(f.read())

vosk_text = json.loads(rec.FinalResult())["text"]

print("Vosk (Kaldi-style) Output:")
print(vosk_text)


Vosk (Kaldi-style) Output:
what's the biggest misunderstanding about you whenever you hear about somebody is


In [7]:
#==========================================
# Model 3 — Kaldi alternative (Wav2Vec 2.0)
#==========================================
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

#=========================================================
#warning removing
import os, warnings, transformers
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
warnings.filterwarnings("ignore")
transformers.logging.set_verbosity_error()
#=========================================================


processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

speech, sr = torchaudio.load(audio_path)
speech = speech.squeeze()

inputs = processor(speech, sampling_rate=sr, return_tensors="pt")

with torch.no_grad():
    logits = model(inputs.input_values).logits

pred_ids = torch.argmax(logits, dim=-1)
wav2vec_text = processor.decode(pred_ids[0]).lower()

print("Wav2Vec 2.0 Output:")
print(wav2vec_text)


Wav2Vec 2.0 Output:
what's the biggest misanderstanding about you whenever you sur about somebodos


In [8]:
#==========================================
# Model 4 — Hugging Face ASR Pipeline
#==========================================
from transformers import pipeline

hf_asr = pipeline(
    "automatic-speech-recognition",
    model="facebook/wav2vec2-base-960h"
)

hf_result = hf_asr(audio_path)
hf_text = hf_result["text"].lower()

print("Hugging Face ASR Output:")
print(hf_text)


Hugging Face ASR Output:
what's the biggest misanderstanding about you whenever you sur about somebodos


# WER = Word Error Rate
It measures how wrong a speech-to-text output is compared to the correct text.

# WER=(S+D+I​) / N

- S = Substitutions (wrong words)

- D = Deletions (missing words)

- I = Insertions (extra words)

- N = Words in reference text

In [9]:
#=========================
# Simple Evaluation (WER)
#=========================
from jiwer import wer

ground_truth = "What's the biggest misunderstanding about you? Whenever you hear about somebody who's..."

print("WER Whisper:", wer(ground_truth, whisper_text))
print("WER Vosk:", wer(ground_truth, vosk_text))
print("WER Wav2Vec:", wer(ground_truth, wav2vec_text))
print("WER HuggingFace:", wer(ground_truth, hf_text))


WER Whisper: 0.0
WER Vosk: 0.3333333333333333
WER Wav2Vec: 0.5833333333333334
WER HuggingFace: 0.5833333333333334


In [10]:
#=========================
# Comparison Summary
#=========================
print("\n===== FINAL COMPARISON =====")
print("Whisper      :", whisper_text)
print("Vosk (Kaldi) :", vosk_text)
print("Wav2Vec 2.0  :", wav2vec_text)
print("HF Pipeline  :", hf_text)



===== FINAL COMPARISON =====
Whisper      :  What's the biggest misunderstanding about you? Whenever you hear about somebody who's...
Vosk (Kaldi) : what's the biggest misunderstanding about you whenever you hear about somebody is
Wav2Vec 2.0  : what's the biggest misanderstanding about you whenever you sur about somebodos
HF Pipeline  : what's the biggest misanderstanding about you whenever you sur about somebodos
